In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Set seed for reproducibility
np.random.seed(42)

# ============================================================================
# PARAMETERS (Table 2)
# ============================================================================
L = 5  # locations
T = 208  # weeks
C = 4  # channels

# Media parameters
beta_media = np.array([1.15, 0.85, 0.90, 0.55])
# beta_media = np.array([1.50, 0.90, 0.70, 0.40])  # Was [1.15, 0.85, 0.90, 0.55]
alpha = np.array([0.40, 0.50, 0.30, 0.60])  # adstock decay
# alpha = np.array([0.70, 0.75, 0.65, 0.80])  # Was [0.40, 0.50, 0.30, 0.60]
K = np.array([8000, 6000, 5000, 4000])  # saturation half-point
# K = np.array([4000, 3000, 2500, 2000])  # Was [8000, 6000, 5000, 4000]
# S = np.array([1.5, 1.8, 1.2, 2.0])  # saturation shape
S = np.array([2.5, 3.0, 2.2, 3.5])  # Was [1.5, 1.8, 1.2, 2.0]

# Other parameters
beta_0 = 7.5
beta_p = -1.2
beta_d = 0.015
beta_u = -0.05
beta_c = 0.01
beta_t = 0.001
beta_s1 = 0.08
beta_s2 = 0.05
gamma = np.array([0.10, 0.15, 0.20])  # holidays
sigma = 0.05

# ============================================================================
# TRANSFORMATION FUNCTIONS
# ============================================================================
def adstock_transform(x, alpha):
    """Geometric adstock transformation"""
    adstocked = np.zeros_like(x)
    adstocked[0] = x[0]
    for t in range(1, len(x)):
        adstocked[t] = x[t] + alpha * adstocked[t-1]
    return adstocked

def hill_saturation(z, K, S):
    """Hill saturation transformation"""
    return (z**S) / (K**S + z**S)

def apply_transformations(spend, alpha, K, S):
    """Apply adstock then saturation"""
    adstocked = adstock_transform(spend, alpha)
    saturated = hill_saturation(adstocked, K, S)
    return saturated

# ============================================================================
# DATA GENERATION
# ============================================================================
# Generate media spend (different patterns per location)
media_spend = np.zeros((T, C, L))
for l in range(L):
    for c in range(C):
        base = 3000 + c * 1000 + l * 500
        trend = np.linspace(0, 500, T) 
        seasonality = 500 * np.sin(2 * np.pi * np.arange(T) / 52) 
        noise = np.random.gamma(2, 200, T) 
        media_spend[:, c, l] = base + trend + seasonality + noise

# Generate control variables
price = 50 + 5 * np.random.randn(T, L) + np.linspace(0, 10, T)[:, None]
discount = 0.05 + 0.02 * np.random.rand(T, L)
unemployment = 5.0 + 0.5 * np.sin(2 * np.pi * np.arange(T) / 52) + 0.2 * np.random.randn(T)
cci = 100 + 5 * np.sin(2 * np.pi * np.arange(T) / 52) + np.random.randn(T)

# Holiday indicators
holidays = np.zeros((T, 3))
holidays[26, 0] = 1  # Independence Day (week 27)
holidays[151, 1] = 1  # Thanksgiving (week 152)
holidays[155, 2] = 1  # Christmas (week 156)

# Time trend
time_trend = np.arange(T)

# Seasonality
seasonality_sin = np.sin(2 * np.pi * np.arange(T) / 52)
seasonality_cos = np.cos(2 * np.pi * np.arange(T) / 52)

# Generate sale units (log scale)
log_sales = np.zeros((T, L))

for l in range(L):
    # Base + location effect
    base = beta_0 + (0.1 * l if l > 0 else 0)
    
    # Media effects (with transformations)
    media_effect = np.zeros(T)
    for c in range(C):
        transformed = apply_transformations(media_spend[:, c, l], alpha[c], K[c], S[c])
        location_coef = beta_media[c] + (0.05 * l if l > 0 else 0)
        media_effect += location_coef * transformed
    
    # All effects combined
    log_sales[:, l] = (
        base +
        media_effect +
        beta_p * np.log(price[:, l]) +
        beta_d * discount[:, l] +
        beta_u * unemployment +
        beta_c * cci +
        beta_t * time_trend +
        gamma[0] * holidays[:, 0] +
        gamma[1] * holidays[:, 1] +
        gamma[2] * holidays[:, 2] +
        beta_s1 * seasonality_sin +
        beta_s2 * seasonality_cos +
        sigma * np.random.randn(T)
    )

# Convert to actual sales
sales = np.exp(log_sales)

# ============================================================================
# TRAIN/TEST SPLIT
# ============================================================================
train_idx = 156
test_idx = T

# ============================================================================
# MODEL A: CORRECT (Adstock + Saturation)
# ============================================================================
def model_a_predict(params, X_spend, X_controls):
    """Model A with adstock and saturation"""
    beta_0_est = params[0]
    beta_media_est = params[1:5]
    alpha_est = params[5:9]
    K_est = params[9:13]
    S_est = params[13:17]
    other_coefs = params[17:]
    
    T_local, L_local = X_controls.shape[0], X_controls.shape[1]
    predictions = np.zeros((T_local, L_local))
    
    for l in range(L_local):
        pred = beta_0_est
        
        # Media effects
        for c in range(C):
            transformed = apply_transformations(X_spend[:, c, l], alpha_est[c], K_est[c], S_est[c])
            pred += beta_media_est[c] * transformed
        
        # Controls
        pred += other_coefs[0] * np.log(X_controls[:, l, 0])  # price
        pred += other_coefs[1] * X_controls[:, l, 1]  # discount
        pred += other_coefs[2] * X_controls[:, l, 2]  # unemployment
        pred += other_coefs[3] * X_controls[:, l, 3]  # cci
        pred += other_coefs[4] * X_controls[:, l, 4]  # time
        pred += other_coefs[5] * X_controls[:, l, 5]  # holiday1
        pred += other_coefs[6] * X_controls[:, l, 6]  # holiday2
        pred += other_coefs[7] * X_controls[:, l, 7]  # holiday3
        pred += other_coefs[8] * X_controls[:, l, 8]  # sin
        pred += other_coefs[9] * X_controls[:, l, 9]  # cos
        
        predictions[:, l] = pred
    
    return predictions

# Prepare training data
X_controls_train = np.zeros((train_idx, L, 10))
for l in range(L):
    X_controls_train[:, l, 0] = price[:train_idx, l]
    X_controls_train[:, l, 1] = discount[:train_idx, l]
    X_controls_train[:, l, 2] = unemployment[:train_idx]
    X_controls_train[:, l, 3] = cci[:train_idx]
    X_controls_train[:, l, 4] = time_trend[:train_idx]
    X_controls_train[:, l, 5] = holidays[:train_idx, 0]
    X_controls_train[:, l, 6] = holidays[:train_idx, 1]
    X_controls_train[:, l, 7] = holidays[:train_idx, 2]
    X_controls_train[:, l, 8] = seasonality_sin[:train_idx]
    X_controls_train[:, l, 9] = seasonality_cos[:train_idx]

X_spend_train = media_spend[:train_idx, :, :]
y_train = log_sales[:train_idx, :]

def loss_model_a(params, X_spend, X_controls, y_true):
    pred = model_a_predict(params, X_spend, X_controls)
    return np.mean((y_true - pred)**2)

# Initialize and fit Model A
init_params_a = np.concatenate([
    [beta_0],
    beta_media,
    alpha,
    K,
    S,
    [beta_p, beta_d, beta_u, beta_c, beta_t] + list(gamma) + [beta_s1, beta_s2]
])

bounds_a = (
    [(None, None)] +  # beta_0
    [(0, None)] * 4 +  # beta_media
    [(0, 0.99)] * 4 +  # alpha
    [(100, 20000)] * 4 +  # K
    [(0.5, 3)] * 4 +  # S
    [(None, None)] * 10  # other coefs
)

# True values:
# beta_media = [5.0, 4.8, 4.1, 2.9]
# alpha = [0.5, 0.6, 0.4, 0.3]
# K = [1000, 1200, 900, 800]
# S = [2.5, 3.0, 2.2, 3.5]  # CORRECTED

print("Fitting Model A (Correct)...")
result_a = minimize(loss_model_a, init_params_a, args=(X_spend_train, X_controls_train, y_train),
                    method='L-BFGS-B', bounds=bounds_a, options={'maxiter': 500})
params_a = result_a.x

# ============================================================================
# MODEL B: MISSPECIFIED (No transformations)
# ============================================================================
def model_b_predict(params, X_spend, X_controls):
    """Model B: linear, no adstock/saturation"""
    beta_0_est = params[0]
    beta_media_est = params[1:5]
    other_coefs = params[5:]
    
    T_local, L_local = X_controls.shape[0], X_controls.shape[1]
    predictions = np.zeros((T_local, L_local))
    
    for l in range(L_local):
        pred = beta_0_est
        
        # Media effects (raw spend, scaled)
        for c in range(C):
            pred += beta_media_est[c] * (X_spend[:, c, l] / 10000)
        
        # Controls
        pred += other_coefs[0] * np.log(X_controls[:, l, 0])
        pred += other_coefs[1] * X_controls[:, l, 1]
        pred += other_coefs[2] * X_controls[:, l, 2]
        pred += other_coefs[3] * X_controls[:, l, 3]
        pred += other_coefs[4] * X_controls[:, l, 4]
        pred += other_coefs[5] * X_controls[:, l, 5]
        pred += other_coefs[6] * X_controls[:, l, 6]
        pred += other_coefs[7] * X_controls[:, l, 7]
        pred += other_coefs[8] * X_controls[:, l, 8]
        pred += other_coefs[9] * X_controls[:, l, 9]
        
        predictions[:, l] = pred
    
    return predictions

def loss_model_b(params, X_spend, X_controls, y_true):
    pred = model_b_predict(params, X_spend, X_controls)
    return np.mean((y_true - pred)**2)

# Initialize and fit Model B
init_params_b = np.concatenate([
    [beta_0],
    beta_media * 2,  # Compensate for no transformation
    [beta_p, beta_d, beta_u, beta_c, beta_t] + list(gamma) + [beta_s1, beta_s2]
])

bounds_b = (
    [(None, None)] +
    [(0, None)] * 4 +
    [(None, None)] * 10
)

print("Fitting Model B (Misspecified)...")
result_b = minimize(loss_model_b, init_params_b, args=(X_spend_train, X_controls_train, y_train),
                    method='L-BFGS-B', bounds=bounds_b, options={'maxiter': 500})
params_b = result_b.x

# ============================================================================
# TEST SET EVALUATION
# ============================================================================
X_controls_test = np.zeros((test_idx - train_idx, L, 10))
for l in range(L):
    X_controls_test[:, l, 0] = price[train_idx:test_idx, l]
    X_controls_test[:, l, 1] = discount[train_idx:test_idx, l]
    X_controls_test[:, l, 2] = unemployment[train_idx:test_idx]
    X_controls_test[:, l, 3] = cci[train_idx:test_idx]
    X_controls_test[:, l, 4] = time_trend[train_idx:test_idx]
    X_controls_test[:, l, 5] = holidays[train_idx:test_idx, 0]
    X_controls_test[:, l, 6] = holidays[train_idx:test_idx, 1]
    X_controls_test[:, l, 7] = holidays[train_idx:test_idx, 2]
    X_controls_test[:, l, 8] = seasonality_sin[train_idx:test_idx]
    X_controls_test[:, l, 9] = seasonality_cos[train_idx:test_idx]

X_spend_test = media_spend[train_idx:test_idx, :, :]
y_test = log_sales[train_idx:test_idx, :]
sales_test = sales[train_idx:test_idx, :]

# Predictions
pred_a_test = model_a_predict(params_a, X_spend_test, X_controls_test)
pred_b_test = model_b_predict(params_b, X_spend_test, X_controls_test)

sales_pred_a = np.exp(pred_a_test)
sales_pred_b = np.exp(pred_b_test)

# Sales metrics
r2_a = r2_score(sales_test.flatten(), sales_pred_a.flatten())
r2_b = r2_score(sales_test.flatten(), sales_pred_b.flatten())

mape_a = mean_absolute_percentage_error(sales_test.flatten(), sales_pred_a.flatten()) * 100
mape_b = mean_absolute_percentage_error(sales_test.flatten(), sales_pred_b.flatten()) * 100

rmse_a = np.sqrt(mean_squared_error(sales_test.flatten(), sales_pred_a.flatten()))
rmse_b = np.sqrt(mean_squared_error(sales_test.flatten(), sales_pred_b.flatten()))

# ============================================================================
# ROI CALCULATION
# ============================================================================
def calculate_roi(params, model_type, X_spend_full, X_controls_full, price_full):
    """Calculate ROI by counterfactual: all channels on vs channel i off"""
    roi_estimates = []
    
    for c in range(C):
        # Scenario 1: All channels active
        if model_type == 'A':
            pred_all = model_a_predict(params, X_spend_full, X_controls_full)
        else:
            pred_all = model_b_predict(params, X_spend_full, X_controls_full)
        
        # Scenario 2: Channel c set to zero
        X_spend_counterfactual = X_spend_full.copy()
        X_spend_counterfactual[:, c, :] = 0
        
        if model_type == 'A':
            pred_without = model_a_predict(params, X_spend_counterfactual, X_controls_full)
        else:
            pred_without = model_b_predict(params, X_spend_counterfactual, X_controls_full)
        
        # Incremental revenue
        incremental_sales = np.exp(pred_all) - np.exp(pred_without)
        incremental_revenue = np.sum(incremental_sales * price_full)
        
        # Total spend
        total_spend = np.sum(X_spend_full[:, c, :])
        
        # ROI
        roi = incremental_revenue / total_spend if total_spend > 0 else 0
        roi_estimates.append(roi)
    
    return np.array(roi_estimates)

# True ROI (using ground truth parameters)
true_params = np.concatenate([
    [beta_0], beta_media, alpha, K, S,
    [beta_p, beta_d, beta_u, beta_c, beta_t] + list(gamma) + [beta_s1, beta_s2]
])

X_controls_full = np.zeros((test_idx - train_idx, L, 10))
for l in range(L):
    X_controls_full[:, l, :] = X_controls_test[:, l, :]

price_full = price[train_idx:test_idx, :]

true_roi = calculate_roi(true_params, 'A', X_spend_test, X_controls_full, price_full)

# Estimated ROI
roi_a = calculate_roi(params_a, 'A', X_spend_test, X_controls_full, price_full)
roi_b = calculate_roi(params_b, 'B', X_spend_test, X_controls_full, price_full)

# ROI metrics
roi_mae_a = np.mean(np.abs(true_roi - roi_a))
roi_mae_b = np.mean(np.abs(true_roi - roi_b))

roi_mape_a = np.mean(np.abs((true_roi - roi_a) / true_roi)) * 100
roi_mape_b = np.mean(np.abs((true_roi - roi_b) / true_roi)) * 100

# ============================================================================
# PRINT RESULTS (Tables 2 & 3)
# ============================================================================
print("\n" + "="*80)
print("TABLE 2: Comparison of Sales Prediction Accuracy and ROI Recovery Accuracy")
print("="*80)
print(f"{'Performance Metric':<40} {'Model A (Correct)':<25} {'Model B (Misspecified)':<25}")
print("-"*80)
print("\nSales Prediction Metrics (Test Set)")
print("-"*80)
print(f"{'R²':<40} {r2_a:<25.2f} {r2_b:<25.2f}")
print(f"{'MAPE (%)':<40} {mape_a:<25.2f} {mape_b:<25.2f}")
print(f"{'RMSE':<40} {rmse_a:<25.2f} {rmse_b:<25.2f}")

print("\nROI Recovery Metrics (Test Set)")
print("-"*80)
print(f"{'Mean Absolute Error':<40} {roi_mae_a:<25.2f} {roi_mae_b:<25.2f}")
print(f"{'MAPE (%)':<40} {roi_mape_a:<25.2f} {roi_mape_b:<25.2f}")
print(f"{'Relative Performance':<40} {'1.0x':<25} {f'{roi_mae_b/roi_mae_a:.1f}x worse':<25}")

print("\nChannel-Level ROI Estimates")
print("-"*80)
for c in range(C):
    print(f"Channel {c+1} (True: {true_roi[c]:.2f})    {roi_a[c]:<25.2f} {roi_b[c]:<25.2f}")

print("\n" + "="*80)
print("CONFLICT DEMONSTRATION:")
print("="*80)
if r2_b > r2_a:
    print(f"✓ Model B has HIGHER R² ({r2_b:.4f} vs {r2_a:.4f})")
else:
    print(f"✗ Model A has higher R² - conflict not achieved")

if roi_mae_b > roi_mae_a:
    print(f"✓ Model B has WORSE ROI recovery ({roi_mae_b:.4f} vs {roi_mae_a:.4f})")
    print(f"✓ Improvement factor: {roi_mae_b/roi_mae_a:.1f}x")
else:
    print(f"✗ Model A has worse ROI - conflict not achieved")
print("="*80)

Fitting Model A (Correct)...
Fitting Model B (Misspecified)...

TABLE 2: Comparison of Sales Prediction Accuracy and ROI Recovery Accuracy
Performance Metric                       Model A (Correct)         Model B (Misspecified)   
--------------------------------------------------------------------------------

Sales Prediction Metrics (Test Set)
--------------------------------------------------------------------------------
R²                                       0.90                      0.93                     
MAPE (%)                                 10.38                     9.07                     
RMSE                                     169.13                    138.55                   

ROI Recovery Metrics (Test Set)
--------------------------------------------------------------------------------
Mean Absolute Error                      3.12                      4.40                     
MAPE (%)                                 141.92                    209.59          

In [2]:
import sys
import numpy
import sklearn
import matplotlib

print(f"Python: {sys.version}")
print(f"NumPy: {numpy.__version__}")
print(f"scikit-learn: {sklearn.__version__}")
print(f"matplotlib: {matplotlib.__version__}")

Python: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
NumPy: 1.26.4
scikit-learn: 1.5.1
matplotlib: 3.9.0


In [3]:
# raise Exception('stop here.')

In [4]:
##### visualization in 2 figures 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['font.size'] = 10

# ============================================================================
# DATA FROM RESULTS
# ============================================================================
# Sales prediction metrics
r2_a, r2_b = 0.90, 0.93
mape_a, mape_b = 10.38, 9.07
rmse_a, rmse_b = 169.13, 138.55

# ROI recovery metrics
roi_mae_a, roi_mae_b = 3.12, 4.40
roi_mape_a, roi_mape_b = 141.92, 209.59

# Channel-level ROI
true_roi = np.array([2.48, 2.44, 2.06, 1.46])
roi_a = np.array([7.88, 5.34, 4.84, 2.87])
roi_b = np.array([8.45, 6.64, 6.24, 4.73])

# ============================================================================
# FIGURE 1: SALES PREDICTION METRICS COMPARISON
# ============================================================================
fig1, axes = plt.subplots(1, 3, figsize=(13, 4.5))

# R² comparison
ax1 = axes[0]
x_pos = [0, 1]
r2_values = [r2_a, r2_b]
colors = ['#3498db', '#e74c3c']
bars1 = ax1.bar(x_pos, r2_values, width=0.2, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('Out-of-Sample R²', fontsize=11, fontweight='bold')
ax1.set_title('(a) R² Comparison\n(Higher is Better)', fontsize=11, fontweight='bold', pad=12)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['Model A\n(Correct)', 'Model B\n(Misspecified)'], fontsize=10)
ax1.set_ylim([0.85, 0.96])
ax1.axhline(y=0.90, color='gray', linestyle='--', alpha=0.3)
for i, (bar, val) in enumerate(zip(bars1, r2_values)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.003, 
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
# Highlight winner - repositioned
ax1.text(1, 0.945, '✓ Winner', ha='center', fontsize=9, color='darkred', fontweight='bold')

# MAPE comparison
ax2 = axes[1]
mape_values = [mape_a, mape_b]
bars2 = ax2.bar(x_pos, mape_values, width=0.2, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('MAPE (%)', fontsize=11, fontweight='bold')
ax2.set_title('(b) MAPE Comparison\n(Lower is Better)', fontsize=11, fontweight='bold', pad=12)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['Model A\n(Correct)', 'Model B\n(Misspecified)'], fontsize=10)
ax2.set_ylim([0, 13])
for i, (bar, val) in enumerate(zip(bars2, mape_values)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.25, 
             f'{val:.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax2.text(1, 11.5, '✓ Winner', ha='center', fontsize=9, color='darkred', fontweight='bold')

# RMSE comparison
ax3 = axes[2]
rmse_values = [rmse_a, rmse_b]
bars3 = ax3.bar(x_pos, rmse_values, width=0.2, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax3.set_ylabel('RMSE', fontsize=11, fontweight='bold')
ax3.set_title('(c) RMSE Comparison\n(Lower is Better)', fontsize=11, fontweight='bold', pad=12)
ax3.set_xticks(x_pos)
ax3.set_xticklabels(['Model A\n(Correct)', 'Model B\n(Misspecified)'], fontsize=10)
ax3.set_ylim([0, 210])
for i, (bar, val) in enumerate(zip(bars3, rmse_values)):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 4, 
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax3.text(1, 180, '✓ Winner', ha='center', fontsize=9, color='darkred', fontweight='bold')

plt.suptitle('Sales Prediction Accuracy: Model B (Misspecified) Wins', 
             fontsize=13, fontweight='bold', y=1.00)
plt.tight_layout()
plt.savefig('figure1_sales_metrics.png', dpi=300, bbox_inches='tight')
print("✓ Saved: figure1_sales_metrics.png")
plt.close()

✓ Saved: figure1_sales_metrics.png


In [5]:
# ============================================================================
# FIGURE 2: ROI RECOVERY PERFORMANCE
# ============================================================================
fig2, axes = plt.subplots(2, 2, figsize=(13, 10.5))

# (a) ROI MAE comparison
ax1 = axes[0, 0]
roi_mae_values = [roi_mae_a, roi_mae_b]
bars1 = ax1.bar(x_pos, roi_mae_values, width=0.2, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax1.set_ylabel('ROI Mean Absolute Error', fontsize=11, fontweight='bold')
ax1.set_title('(a) ROI Recovery MAE\n(Lower is Better)', fontsize=11, fontweight='bold', pad=12)
ax1.set_xticks(x_pos)
ax1.set_xticklabels(['Model A\n(Correct)', 'Model B\n(Misspecified)'], fontsize=10)
ax1.set_ylim([0, 5.5])
for i, (bar, val) in enumerate(zip(bars1, roi_mae_values)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.12, 
             f'{val:.2f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax1.text(0, 3.5, '✓ Winner', ha='center', fontsize=9, color='darkblue', fontweight='bold')
ax1.axhline(y=roi_mae_a, color='blue', linestyle='--', alpha=0.3, label='Model A level')

# (b) ROI MAPE comparison
ax2 = axes[0, 1]
roi_mape_values = [roi_mape_a, roi_mape_b]
bars2 = ax2.bar(x_pos, roi_mape_values, width=0.2, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('ROI MAPE (%)', fontsize=11, fontweight='bold')
ax2.set_title('(b) ROI Recovery MAPE\n(Lower is Better)', fontsize=11, fontweight='bold', pad=12)
ax2.set_xticks(x_pos)
ax2.set_xticklabels(['Model A\n(Correct)', 'Model B\n(Misspecified)'], fontsize=10)
ax2.set_ylim([0, 260])
for i, (bar, val) in enumerate(zip(bars2, roi_mape_values)):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 6, 
             f'{val:.2f}%', ha='center', va='bottom', fontsize=10, fontweight='bold')
ax2.text(0, 165, '✓ Winner', ha='center', fontsize=9, color='darkblue', fontweight='bold')
improvement = roi_mae_b / roi_mae_a
ax2.text(0.5, 240, f'Improvement: {improvement:.1f}× better', ha='center', 
         fontsize=10, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5), fontweight='bold')

# (c) Channel-level ROI estimates - NARROWER BARS
ax3 = axes[1, 0]
channels = np.arange(1, 5)
width = 0.18  # Reduced from 0.25 to make bars slimmer
x = np.arange(len(channels))

bars_true = ax3.bar(x - width, true_roi, width, label='True ROI', 
                     color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=1.5)
bars_a = ax3.bar(x, roi_a, width, label='Model A (Correct)', 
                  color='#3498db', alpha=0.7, edgecolor='black', linewidth=1.5)
bars_b = ax3.bar(x + width, roi_b, width, label='Model B (Misspecified)', 
                  color='#e74c3c', alpha=0.7, edgecolor='black', linewidth=1.5)

ax3.set_xlabel('Channel', fontsize=11, fontweight='bold')
ax3.set_ylabel('ROI', fontsize=11, fontweight='bold')
ax3.set_title('(c) Channel-Level ROI Estimates', fontsize=11, fontweight='bold', pad=12)
ax3.set_xticks(x)
ax3.set_xticklabels([f'Ch {i}' for i in channels])
ax3.legend(loc='upper right', fontsize=9, framealpha=0.9)
ax3.set_ylim([0, 10.5])
ax3.grid(axis='y', alpha=0.3)

# Add value labels on top of bars (optional, for clarity)
for bars in [bars_true, bars_a, bars_b]:
    for bar in bars:
        height = bar.get_height()
        ax3.text(bar.get_x() + bar.get_width()/2., height + 0.15,
                f'{height:.2f}', ha='center', va='bottom', fontsize=7.5)

# (d) ROI Recovery Scatter Plot - IMPROVED ANNOTATIONS
ax4 = axes[1, 1]
max_roi = max(true_roi.max(), roi_a.max(), roi_b.max()) + 1
ax4.plot([0, max_roi], [0, max_roi], 'k--', alpha=0.3, linewidth=2, label='Perfect Recovery')

ax4.scatter(true_roi, roi_a, s=150, alpha=0.7, color='#3498db', 
            edgecolor='black', linewidth=2, marker='o', label='Model A (Correct)', zorder=3)
ax4.scatter(true_roi, roi_b, s=150, alpha=0.7, color='#e74c3c', 
            edgecolor='black', linewidth=2, marker='s', label='Model B (Misspecified)', zorder=3)

# Improved channel annotations - prevent overlap
annotation_offsets_a = [(8, 5), (8, 5), (8, 5), (8, 5)]
annotation_offsets_b = [(8, -15), (8, -15), (8, -15), (8, -15)]

for i in range(len(true_roi)):
    ax4.annotate(f'Ch{i+1}', (true_roi[i], roi_a[i]), 
                xytext=annotation_offsets_a[i], textcoords='offset points', 
                fontsize=8, color='darkblue', fontweight='bold')
    ax4.annotate(f'Ch{i+1}', (true_roi[i], roi_b[i]), 
                xytext=annotation_offsets_b[i], textcoords='offset points', 
                fontsize=8, color='darkred', fontweight='bold')

ax4.set_xlabel('True ROI', fontsize=11, fontweight='bold')
ax4.set_ylabel('Estimated ROI', fontsize=11, fontweight='bold')
ax4.set_title('(d) ROI Recovery: Estimated vs True', fontsize=11, fontweight='bold', pad=12)
ax4.legend(loc='upper center', fontsize=9, framealpha=0.9)
ax4.grid(alpha=0.3)
ax4.set_xlim([0, max_roi])
ax4.set_ylim([0, max_roi])

# Add text box
textstr = 'Model A clusters closer\nto perfect recovery line'
props = dict(boxstyle='round', facecolor='lightblue', alpha=0.5)
ax4.text(0.98, 0.02, textstr, transform=ax4.transAxes, fontsize=9,
         verticalalignment='bottom', horizontalalignment='right', bbox=props)

plt.suptitle('ROI Recovery Performance: Model A (Correct) Wins Despite Lower R²', 
             fontsize=13, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('figure2_roi_recovery.png', dpi=300, bbox_inches='tight')
print("✓ Saved: figure2_roi_recovery.png")
plt.close()


✓ Saved: figure2_roi_recovery.png
